In [13]:
import torch
import torchvision.models as models
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter


In [2]:
# train_dataset = CustomDataset()
# val_dataset = CustomDataset()

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [27]:
# Define the transformations to apply to the CIFAR-10 data
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

download = True
# Define the training and test datasets
train_dataset = datasets.Flowers102(root='./data', split="train", download=download, transform=data_transforms)
test_dataset = datasets.Flowers102(root='./data', split="test", download=download, transform=data_transforms)

# Define the dataloaders to load the data in batches during training and testing
batch_size = 64

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


In [28]:
print(f"Number of training batches: {len(train_loader)}")
print(f"Number of testing batches: {len(test_loader)}")

Number of training batches: 16
Number of testing batches: 97


In [29]:
num_params = sum(torch.numel(p) for p in model.parameters())
print(f"Number of parameters: {num_params}")

Number of parameters: 11228838


In [8]:
def model_eval(model, dataloader):
    
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

In [16]:
log_dir = "./logs/baseline"  # Set the directory for storing the logs
writer = SummaryWriter(log_dir)

# the network to test
model = models.resnet18(pretrained=False)
num_classes = 102

learning_rate = 1e-3

# Modify the last fully connected layer
fc_input = model.fc.in_features
model.fc = nn.Linear(fc_input, num_classes)

# Step 5: Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

num_epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

log_train_every = 1
log_test_every = 5

for epoch in range(num_epochs):
    running_loss = 0.0
    
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    # compute training & testing accuracy every couple of iterations        
    if (epoch+1) % log_train_every == 0:
        train_accuracy = model_eval(model, train_loader)

        # Log the loss
        writer.add_scalar('Loss/train', loss.cpu().item(), epoch)
        
        # Log epoch loss
        writer.add_scalar('Epoch Loss/train', running_loss, epoch)

        # Log the training accuracy
        writer.add_scalar('Accuracy/train', train_accuracy, epoch)
        print(f"Epoch {epoch+1} : Loss: {running_loss} Accuracy: {train_accuracy}")
        
    if (epoch+1) % log_test_every == 0:
        test_accuracy = model_eval(model, test_loader)

        # Log the test accuracy
        writer.add_scalar('Accuracy/test', test_accuracy, epoch)
        print(f"Epoch {epoch+1} : Test Accuracy: {test_accuracy}")
        
writer.close()
print("Training finished.")

Epoch 1 : Loss: 71.29615211486816 Accuracy: 3.9215686274509802
Epoch 2 : Loss: 86.322012424469 Accuracy: 1.7647058823529411
Epoch 3 : Loss: 72.76761102676392 Accuracy: 2.549019607843137
Epoch 4 : Loss: 70.11820840835571 Accuracy: 2.3529411764705883
Epoch 5 : Loss: 68.54058790206909 Accuracy: 4.117647058823529
Epoch 5 : Test Accuracy: 2.406895430151244
Epoch 6 : Loss: 66.04335927963257 Accuracy: 4.607843137254902
Epoch 7 : Loss: 63.95159363746643 Accuracy: 6.666666666666667
Epoch 8 : Loss: 62.56390738487244 Accuracy: 9.215686274509803
Epoch 9 : Loss: 60.95228910446167 Accuracy: 10.0
Epoch 10 : Loss: 59.11158847808838 Accuracy: 13.137254901960784
Epoch 10 : Test Accuracy: 7.643519271426248
Epoch 11 : Loss: 57.35419273376465 Accuracy: 13.72549019607843
Epoch 12 : Loss: 55.76991415023804 Accuracy: 13.333333333333334
Epoch 13 : Loss: 53.62331509590149 Accuracy: 18.431372549019606
Epoch 14 : Loss: 52.91764545440674 Accuracy: 19.901960784313726
Epoch 15 : Loss: 50.84020972251892 Accuracy: 18.

## Curriculum Learning experiment

So.. The curriculum for the following training procedure is that we increase the number of classes progressively in the dataset. All original labels are shited by 1. So, label 0 becomes 1, 1 becomes 2.. and so on. 
We first start with 2 classes 0 and 1 where 0 represents rest of classes and 1 represents the class that was originally 0. Then we work with 3 classes after 2 epochs: 0(rest of the classes), 1 and 2.

In [23]:
class SubsetDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, classes, num_classes, train=True, total_classes=102):
        self.dataset = dataset
        self.classes = classes
        self.num_classes = num_classes
        self.train = train
        self.total_classes = total_classes

    def __getitem__(self, index):
        image, label = self.dataset[index]
        if not self.train:
            return image, (label+1)%self.total_classes# we would reserve 0 for non-existing classes
        
        if label in self.classes[:self.num_classes]:
            return image, (label+1)%self.total_classes
        else:
            return torch.zeros_like(image), 0# IMPORTANT CHOICE MADE HERE 

    def __len__(self):
        return len(self.dataset)

In [30]:
increment_step = 2

log_dir = f"./logs/curriculum_step{increment_step}"  # Set the directory for storing the logs
writer = SummaryWriter(log_dir)

####### Reset the model
# the network to test
model = models.resnet18(pretrained=False)
num_classes = 102

learning_rate = 1e-3

# Modify the last fully connected layer
fc_input = model.fc.in_features
model.fc = nn.Linear(fc_input, num_classes)

# print(model)

# Step 5: Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

##################

num_epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

curriculum_epochs = 5
nclass = increment_step


log_train_every = 1
log_test_every = 5

test_dataset = SubsetDataset(test_dataset, list(range(102)), 102, train=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=4, shuffle=True)

for epoch in range(num_epochs):
    running_loss = 0.0
    
    subset_classes = list(range(nclass))

    # Create the modified dataset
    subset_dataset = SubsetDataset(train_dataset, list(range(102)), nclass)
    print(f"Showing a total of {len(subset_dataset.classes[:subset_dataset.num_classes])} classes now")
    

    # Create the train loader using the modified dataset
    train_loader = torch.utils.data.DataLoader(subset_dataset, batch_size=batch_size, num_workers=4, shuffle=True)

    
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

#         if (i+1) % 100 == 0:
#             print(f"[Epoch: {epoch + 1}, Batch: {i + 1}] Loss: {running_loss/100:.3f}")
#             running_loss = 0.0
        
    # compute training & testing accuracy every couple of iterations        
    if (epoch+1) % log_train_every == 0:
        train_accuracy = model_eval(model, train_loader)

        # Log the loss
        writer.add_scalar('Loss/train', loss.cpu().item(), epoch)
        
        # Log epoch loss
        writer.add_scalar('Epoch Loss/train', loss.cpu().item(), epoch)

        # Log the training accuracy
        writer.add_scalar('Accuracy/train', train_accuracy, epoch)
        
        writer.add_scalar('Classes Shown/train', len(subset_dataset.classes[:subset_dataset.num_classes]), epoch)
        
        print(f"Epoch {epoch+1} : Loss: {running_loss} Accuracy: {train_accuracy}")
        
    if (epoch+1) % log_test_every == 0:
        test_accuracy = model_eval(model, test_loader)

        # Log the test accuracy
        writer.add_scalar('Accuracy/test', test_accuracy, epoch)
        print(f"Epoch {epoch+1} : Test Accuracy: {test_accuracy}")
    
    nclass += increment_step

writer.close()
print("Training finished.")

Showing a total of 2 classes now
Epoch 1 : Loss: 20.042649937968235 Accuracy: 99.01960784313725
Showing a total of 4 classes now
Epoch 2 : Loss: 18.17356050433591 Accuracy: 97.05882352941177
Showing a total of 6 classes now
Epoch 3 : Loss: 2.576610756213995 Accuracy: 95.09803921568627
Showing a total of 8 classes now
Epoch 4 : Loss: 3.7195497304201126 Accuracy: 93.13725490196079
Showing a total of 10 classes now
Epoch 5 : Loss: 4.628831289708614 Accuracy: 92.15686274509804
Epoch 5 : Test Accuracy: 1.073345259391771
Showing a total of 12 classes now
Epoch 6 : Loss: 5.729439575225115 Accuracy: 90.19607843137256
Showing a total of 14 classes now
Epoch 7 : Loss: 6.74860156327486 Accuracy: 87.6470588235294
Showing a total of 16 classes now
Epoch 8 : Loss: 7.499422624707222 Accuracy: 86.07843137254902
Showing a total of 18 classes now
Epoch 9 : Loss: 8.59327507019043 Accuracy: 85.3921568627451
Showing a total of 20 classes now
Epoch 10 : Loss: 9.609289467334747 Accuracy: 83.33333333333333
Ep

Epoch 79 : Loss: 0.002627466310514137 Accuracy: 100.0
Showing a total of 102 classes now
Epoch 80 : Loss: 0.002432374851196073 Accuracy: 100.0
Epoch 80 : Test Accuracy: 21.82468694096601
Showing a total of 102 classes now
Epoch 81 : Loss: 0.0022675123836961575 Accuracy: 100.0
Showing a total of 102 classes now
Epoch 82 : Loss: 0.002122729376424104 Accuracy: 100.0
Showing a total of 102 classes now
Epoch 83 : Loss: 0.0019864468413288705 Accuracy: 100.0
Showing a total of 102 classes now
Epoch 84 : Loss: 0.0018731639283942059 Accuracy: 100.0
Showing a total of 102 classes now
Epoch 85 : Loss: 0.0017652941787673626 Accuracy: 100.0
Epoch 85 : Test Accuracy: 21.987315010570825
Showing a total of 102 classes now
Epoch 86 : Loss: 0.0016702525172149763 Accuracy: 100.0
Showing a total of 102 classes now
Epoch 87 : Loss: 0.0015882783372944687 Accuracy: 100.0
Showing a total of 102 classes now
Epoch 88 : Loss: 0.0015035276119306218 Accuracy: 100.0
Showing a total of 102 classes now
Epoch 89 : Los